In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

In [22]:
import string
punctuations = string.punctuation

def create_freq_dict(text_string):
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string.lower())
    ps = PorterStemmer()
    # remove punctuations
    word_frequencies = dict()
    for word in words:
        word = ps.stem(word)
        if ((word in stopWords) or (word in punctuations)):
            continue
        if word in word_frequencies:
            word_frequencies[word] += 1
        else:
            word_frequencies[word] = 1

    return word_frequencies

In [23]:
mytext = """An old man lived in the village. 
He was one of the most unfortunate people in the world.
The whole village was tired of him; he was always gloomy, he constantly complained and was always in a bad mood.
The longer he lived, the more bile he was becoming and the more poisonous were his words.
People avoided him, because his misfortune became contagious. It was even unnatural and insulting to be happy next to him."""

mywordfreq = create_freq_dict(mytext)
print(mywordfreq)

{'old': 1, 'man': 1, 'live': 2, 'villag': 2, 'wa': 6, 'one': 1, 'unfortun': 1, 'peopl': 2, 'world': 1, 'whole': 1, 'tire': 1, 'alway': 2, 'gloomi': 1, 'constantli': 1, 'complain': 1, 'bad': 1, 'mood': 1, 'longer': 1, 'bile': 1, 'becom': 1, 'poison': 1, 'hi': 2, 'word': 1, 'avoid': 1, 'becaus': 1, 'misfortun': 1, 'becam': 1, 'contagi': 1, 'even': 1, 'unnatur': 1, 'insult': 1, 'happi': 1, 'next': 1}


In [24]:
def score_sentences(sentences, word_frequencies):
    sentence_scores = dict()

    for sentence in sentences:
        word_count = (len(word_tokenize(sentence)))
        word_count_except_sw = 0
        for word in word_frequencies:
            if word in sentence.lower():
                word_count_except_sw += 1
                #if sentence[:10] in sentence_scores:
                if sentence in sentence_scores:
                    sentence_scores[sentence] += word_frequencies[word]
                else:
                    #sentence_scores[sentence[:10]] = word_frequencies[word]
                    sentence_scores[sentence] = word_frequencies[word]

        #if sentence[:10] in sentence_scores:
        if sentence in sentence_scores:
            # higher no. of words in a sentence should not make its score higher
            #sentence_scores[sentence[:10]] = sentence_scores[sentence[:10]] / word_count_except_sw  
            sentence_scores[sentence] = sentence_scores[sentence] / word_count_except_sw

    return sentence_scores

In [5]:
mytext = """An old man lived in the village. 
He was one of the most unfortunate people in the world.
The whole village was tired of him; he was always gloomy, he constantly complained and was always in a bad mood.
The longer he lived, the more bile he was becoming and the more poisonous were his words.
People avoided him, because his misfortune became contagious. It was even unnatural and insulting to be happy next to him."""
mysentences = sent_tokenize(mytext)
mysentscores = score_sentences(mysentences, mywordfreq)
print(mysentscores)

{'An old man lived in the village.': 1.5, 'He was one of the most unfortunate people in the world.': 2.2, 'The whole village was tired of him; he was always gloomy, he constantly complained and was always in a bad mood.': 1.8888888888888888, 'The longer he lived, the more bile he was becoming and the more poisonous were his words.': 1.875, 'People avoided him, because his misfortune became contagious.': 1.2857142857142858, 'It was even unnatural and insulting to be happy next to him.': 2.0}


In [25]:
mysentscores.keys()

dict_keys(['An old man lived in the village.', 'He was one of the most unfortunate people in the world.', 'The whole village was tired of him; he was always gloomy, he constantly complained and was always in a bad mood.', 'The longer he lived, the more bile he was becoming and the more poisonous were his words.', 'People avoided him, because his misfortune became contagious.', 'It was even unnatural and insulting to be happy next to him.'])

In [7]:
def find_average_score(sentence_scores):
    sumValues = 0
    for sentence in sentence_scores:
        sumValues += sentence_scores[sentence]

    # Average value of a sentence from original text
    average = (sumValues / len(sentence_scores))

    return average

In [8]:
myavg = find_average_score(mysentscores)
myavg

1.7916005291005292

In [26]:
def generate_summary(sentences, sentence_scores, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence in sentence_scores and sentence_scores[sentence] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [27]:

mysummary = generate_summary(mysentences, mysentscores, myavg*1.1)
mysummary

' He was one of the most unfortunate people in the world. It was even unnatural and insulting to be happy next to him.'

In [12]:
def run_summarization(text, threshold=1.3):

    word_frequencies = create_freq_dict(text)
    sentences = sent_tokenize(text)
    sentence_scores = score_sentences(sentences, word_frequencies)
    sentence_avg_score = find_average_score(sentence_scores)
    summary = generate_summary(sentences, sentence_scores, threshold * sentence_avg_score)

    return summary

In [15]:
import warnings
warnings.filterwarnings("ignore")



In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [17]:
text="""
An old man lived in the village. He was one of the most unfortunate people in the world. The whole village was tired of him; he was always gloomy, he constantly complained and was always in a bad mood.
The longer he lived, the more bile he was becoming and the more poisonous were his words. People avoided him, because his misfortune became contagious. It was even unnatural and insulting to be happy next to him.

He created the feeling of unhappiness in others.
But one day, when he turned eighty years old, an incredible thing happened. Instantly everyone started hearing the rumour: “An Old Man is happy today, he doesn’t complain about anything, smiles, and even his face is freshened up.”
The whole village gathered together. The old man was asked:
Villager: What happened to you?
“Nothing special. Eighty years I’ve been chasing happiness, and it was useless. And then I decided to live without happiness and just enjoy life. That’s why I’m happy now.” – An Old Man

"""

In [18]:
result = run_summarization(text, 1)
print(result)

 
An old man lived in the village. He was one of the most unfortunate people in the world. The whole village was tired of him; he was always gloomy, he constantly complained and was always in a bad mood. It was even unnatural and insulting to be happy next to him. The old man was asked:
Villager: What happened to you? Eighty years I’ve been chasing happiness, and it was useless. That’s why I’m happy now.” – An Old Man


In [19]:
text = """
In a move to encourage digital payments in the country, the Central Board of Direct Taxes (CBDT) said that banks can not levy any extra charge on transactions made through electronic modes on or after January 1, 2020. "... based on section 10A of the Payment and Settlement Systems Act 2007, any charge including the MDR (Merchant Discount Rate) shall not be applicable on or after 1st January, 2020 on payment made through prescribed electronic modes," CBDT said in a statement.
The electronics modes of transactions include 1) Debit card powered by RuPay, 2) Unified Payment Interface (UPI), 3) Unified Payment Interface Quick Response code (UPI QR code), BHIM UPI QR code. "According to Section 10 A under the Payments and Settlement Systems Act 2007, no bank or system provider shall impose any charge on a payer making payment or beneficiary receiving payment, through electronic modes prescribed under section 269SU of IT Act," the CBDT said.
The regulator noticed that certain banks collect charges for UPI transactions. A certain number of transactions are free and beyond the limit, bank charges for every UPI payment. "Such practice on parts of banks is a breach of Section 10A of the PSS Act as well as section 269SU of the IT act," the CBDT said in its circular. Hence, the banks can not levy any extra charges for UPI transactions.

"Such breach attracts penal provisions under section 271DB of the IT Act as well as section 26 of the PSS Act," CBDT further added.
The banks were also advised to refund charges collected from customers for digital transaction on or after January 1, 2020. "Banks are therefore, advised to immediately refund the charges collected, if any, on or after 1st January, 2020 on transactions carried out using the electronic modes prescribed under section 269SU and not to impose charges on any future transactions carried through the said prescribed mode," the regulator mentioned in its circular.
UPI-based payments clocked 1.49 billion transactions in July, according data released by National Payments Corp. of India. Digital payments have become a necessity in the wake of coronavirus outbreak. "Payment systems have demonstrated that they are dependable and durable, and continue to command a high level of confidence from the general population," said PwC in its report.
With a majority of the sectors that contribute to digital payments still in a state of flux, it is still too early to ascertain the long-term impact of COVID-19 on digital payments, it added.
"""

In [20]:
result = run_summarization(text, 1.2)
print(result)

 The regulator noticed that certain banks collect charges for UPI transactions. A certain number of transactions are free and beyond the limit, bank charges for every UPI payment. Hence, the banks can not levy any extra charges for UPI transactions.
